# QG Height Tendency Equation

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2008, 2, 9, 18)

ds = xr.open_dataset('http://www.ncei.noaa.gov/thredds/dodsC/gfs-g4-anl-files/'
                     f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2')

#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

lats = ds.lat.sel(lat=slice(60, 10)).values
lons = ds.lon.sel(lon=slice(360-160, 360-50)).values

dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

subset = dict(vertical=500 * units.hPa, time=date, lat=slice(60, 10), lon=slice(360-160, 360-50))
hght_500 = ds['Geopotential_height_isobaric'].metpy.sel(subset).values
uwnd_500 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_500 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
subset['vertical'] = 700 * units.hPa
tmpk_700 = (ds['Temperature_isobaric'].metpy.sel(subset).values) * units.K
uwnd_700 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_700 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
subset['vertical'] = 300 * units.hPa
tmpk_300 = (ds['Temperature_isobaric'].metpy.sel(subset).values) * units.K
uwnd_300 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_300 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')

vtime = ds.Geopotential_height_isobaric.time.values[0].astype('datetime64[ms]').astype('O')

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
lons, lats = np.meshgrid(lons, lats)
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

In [ ]:
hght_500s = mpcalc.smooth_n_point(hght_500, 9, 80)
uwnd_500s = mpcalc.smooth_n_point(uwnd_500, 9, 80)
vwnd_500s = mpcalc.smooth_n_point(vwnd_500, 9, 80)

tmpk_700s = mpcalc.smooth_n_point(tmpk_700, 9, 80)
uwnd_700s = mpcalc.smooth_n_point(uwnd_700, 9, 80)
vwnd_700s = mpcalc.smooth_n_point(vwnd_700, 9, 80)

tmpk_300s = mpcalc.smooth_n_point(tmpk_300, 9, 80)
uwnd_300s = mpcalc.smooth_n_point(uwnd_300, 9, 80)
vwnd_300s = mpcalc.smooth_n_point(vwnd_300, 9, 80)

In [ ]:
sigma = 2.0e-6 * units('m^2 Pa^-2 s^-2')
f0 = 1e-4 * units('s^-1')
Rd = mpconstants.Rd

avor_500 = mpcalc.absolute_vorticity(uwnd_500s, vwnd_500s, dx, dy, lats * units('degrees'))
vortadv_500 = mpcalc.advection(avor_500, (uwnd_500s, vwnd_500s), (dx, dy)).to_base_units()

term_A = f0 * vortadv_500

tadv_700 = mpcalc.advection(tmpk_700s, (uwnd_700s, vwnd_700s), (dx, dy)).to_base_units()
tadv_300 = mpcalc.advection(tmpk_300s, (uwnd_300s, vwnd_300s), (dx, dy)).to_base_units()

diff_tadv = ((Rd/(700 * units.hPa)*tadv_700 - Rd/(300 * units.hPa)*tadv_300)/(400 * units.hPa)).to_base_units()

term_B = (-f0**2/sigma*diff_tadv).to_base_units()

In [ ]:
clevs_700_tmpc = np.arange(-40, 41, 2)
clevs_500_hght = np.arange(0, 8000, 60)
clevs_QGHT = np.arange(-8, 8.5, 0.5)

wind_slice = (slice(None, None, 5), slice(None, None, 5))

In [ ]:
# 1st image
area = [-130, -72, 20, 55]
fig = plt.figure(1, figsize=(17,15))
ax1 = plt.subplot(111, projection=mapcrs)
ax1.set_extent(area, ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax1.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax1.contourf(clons, clats, -term_A*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax1.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax1.barbs(lons[wind_slice], lats[wind_slice],
          uwnd_500s.to('kt')[wind_slice].m, vwnd_500s[wind_slice].to('kt').m, pivot='middle',
          color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted QGHT local Abs. Vort. Adv. '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

#plt.savefig(f'QGHT_Vort_{date:%Y%m%d_%H}00.png')
plt.show()

# 2nd image
fig = plt.figure(1, figsize=(17,15))
ax2 = plt.subplot(111, projection=mapcrs)
ax2.set_extent(area, ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax2.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax2.contourf(clons, clats, -term_B*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax2.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax2.barbs(lons[wind_slice], lats[wind_slice],
         uwnd_500s.to('kt')[wind_slice].m, vwnd_500s[wind_slice].to('kt').m, pivot='middle',
         color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted QGHT diff. Temp. Adv. '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

#plt.savefig(f'QGHT_Temp_{date:%Y%m%d_%H}00.png')
plt.show()

# 3rd image
fig = plt.figure(1, figsize=(17,15))
ax3 = plt.subplot(111, projection=mapcrs)
ax3.set_extent(area, ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax3.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax3.contourf(clons, clats, -(term_A+term_B)*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax3.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax3.barbs(lons[wind_slice], lats[wind_slice],
         uwnd_500s.to('kt')[wind_slice].m, vwnd_500s[wind_slice].to('kt').m, pivot='middle',
         color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted Total QGHT (Traditional) '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

#plt.savefig(f'QGHT_Total_{date:%Y%m%d_%H}00.png'.)
plt.show()